## 가볍게 model test

### import

In [4]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import re
fe = fm.FontEntry(fname=r'font/NanumGothic.otf', name='NanumBarunGothic')
fm.fontManager.ttflist.insert(0, fe)
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'})
plt.rc('font', family='NanumBarunGothic')


# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')


from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Model
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import LinearRegression
import lightgbm as lgb


from sklearn import metrics
from sklearn.metrics import mean_squared_error as MSE

import eli5
from eli5.sklearn import PermutationImportance

# 컬럼 화면 설정
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 50)

In [4]:
def get_scores(model, x_train, x_test, y_train, y_test):
    y_pred1 = model.predict(x_train)
    y_pred2 = model.predict(x_test)

    # 음수값 보정 (amount : 음수가 나오지 않는 변수)
    #y_pred1 = np.where(y_pred1 < 0, 69, y_pred1)   # Y.min()
    #y_pred2 = np.where(y_pred2 < 0, 69, y_pred2)   # Y.min()
    y_pred1 = np.where(y_pred1 < 0, 0, y_pred1)   # 0
    y_pred2 = np.where(y_pred2 < 0, 0, y_pred2)   # 0
    #y_pred1 = np.where(y_pred1 < 0, -y_pred1, y_pred1)   # 절댓값
    #y_pred2 = np.where(y_pred2 < 0, -y_pred2, y_pred2)   # 절댓값

    A = r2_score(y_train, y_pred1)
    B = r2_score(y_test, y_pred2)
    C = MSE(y_train, y_pred1) ** 0.5
    D = MSE(y_test, y_pred2) ** 0.5
    return f'r2: {A:.4f} {B:.4f}  rmse: {C:.4f} {D:.4f}'

In [ ]:
# [2] 데이터 탐색 (XY.info(), X_submission.info()) 결측치, 컬럼 dtype
#XY.info()  # 환불금액 - 결측치, 주구매상품, 주구매지점 - object
obj_columns = XY.select_dtypes(include=['object']).columns
#print(obj_columns)
#print(XY.nunique())

# [3] 데이터 전처리
# [3-1] X, X_submission -> X_all
# [3-2] X_all : 컬럼제거, 컬럼 dtype 변경(컬럼의 값을 대체), Encoding(범주형->수치형)
# [3-3] X_all : Scaling (안함, MinMaxScaler, StandardScaler, ...)
# [3-4] X_all -> X, X_submission 분리

X_all = pd.concat([X, X_submission], axis=0)
# 컬럼제거 : 회원ID
# Encoding : obj_columns -> LabelEncoding
X_all = X_all.drop(columns=['회원ID'])
#X_all['주구매상품'] = LabelEncoder().fit_transform(X_all['주구매상품'])
#X_all['주구매지점'] = LabelEncoder().fit_transform(X_all['주구매지점'])

for colname in obj_columns:
    X_all[colname] = LabelEncoder().fit_transform(X_all[colname])

X_all['환불금액'] = X_all['환불금액'].fillna(0)

if False:
    temp = MinMaxScaler().fit_transform(X_all)
    X_all = pd.DataFrame(temp, columns=X_all.columns)
#print(type(temp))  # <class 'numpy.ndarray'>
#print(type(X_all))

#X_all.info()

X = X_all.iloc[:len(X), :]
X_submission = X_all.iloc[len(X):, :]
#print(X.shape, X_submission.shape)  # (3500, 9) (2482, 9)

# [4] 모델링
# [4-1] train_test_split : (X, Y) -> (x_train, x_test, y_train, y_test)
# [4-2] 모델객체 생성, 학습 (x_train, y_train)
# [4-3] 평가 (x_train, y_train), (x_test, y_test)

temp = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=1234)
x_train, x_test, y_train, y_test = temp
print([x.shape for x in temp])  # [(2800, 9), (700, 9), (2800,), (700,)]

# acc: 0.6239 0.6243 AUC: 0.6376 0.6348
#model1 = LogisticRegression().fit(x_train, y_train)
#print(get_scores(model1, x_train, x_test, y_train, y_test))

# acc: 0.6489 0.6571 AUC: 0.6599 0.6591
#model2 = DecisionTreeClassifier(max_depth=3, random_state=0).fit(x_train, y_train)
#print(get_scores(model2, x_train, x_test, y_train, y_test))
#print(model2.get_depth())

#for d in range(2, 14):
#    model2 = DecisionTreeClassifier(max_depth=d, random_state=0).fit(x_train, y_train)
#    print(d, get_scores(model2, x_train, x_test, y_train, y_test))

model3 = RandomForestClassifier(max_depth=5, random_state=0).fit(x_train, y_train)
print(get_scores(model3, x_train, x_test, y_train, y_test))
# acc: 0.6643 0.6557 AUC: 0.6849 0.6681

# [5] 최종모델 선택, 예측값(X_submission), 제출파일생성

fmodel = model3
y_pred = fmodel.predict(X_submission)
pd.DataFrame({'pred': y_pred}).to_csv('result.csv', index=False)

# [6] 제출한 파일 확인
temp = pd.read_csv('result.csv')
print(temp.shape)  # (2482, 1)
#print(temp.tail(10))
print(Y.value_counts(normalize=True))
print(temp['pred'].value_counts(normalize=True))

In [574]:
data_path = '../data/'
df = pd.read_csv(f'{data_path}trainset_240716.csv')

In [575]:
addr = pd.read_csv(f'{data_path}addr_up_ej.csv')

In [576]:
df2 = pd.concat([df, addr], axis=1)

In [577]:
elemSchool_for_train = pd.read_csv(f'{data_path}elemSchool_for_train.csv')
elemSchool_for_test = pd.read_csv(f'{data_path}elemSchool_for_test.csv')

In [578]:
elemSchool_for_train.head(2)
elemSchool_for_train.shape # elemSchool_for_train.head(2)
elemSchool_for_test.shape # (9272, 2)
school_df = pd.concat([elemSchool_for_train, elemSchool_for_test], axis=0)
school_df.shape
school_df.reset_index(drop=True, inplace=True)

In [579]:
df = pd.concat([df2, school_df], axis=1)


In [580]:
subway_train = pd.read_csv(f'{data_path}train_with_subway_infos.csv')
subway_test = pd.read_csv(f'{data_path}test_with_subway_infos.csv')

In [581]:
print(subway_train.shape)
subway_train.head(1)
print(subway_test.shape)
subway_test.head(1)
subway_df = pd.concat([subway_train, subway_test], axis=0)
subway_df.isna().sum()
subway_df.reset_index(drop=True, inplace=True)

(1118822, 18)
(9272, 18)


In [582]:
df = pd.concat([df, subway_df], axis=1)

In [583]:
# df.to_csv(f'{data_path}240717_train.csv', index=False)

In [148]:
df.isna().sum()[100:]

공시가격기반단지명           344872
동수_y                344872
세대수                 344872
사용승인일자              344872
소형학원수               344872
중형학원수               344872
대형학원수               344872
종합학원수               344872
입시학원수               344872
외국어학원수              344872
예체능학원수              344872
독서실수                344872
정보학원수               344872
특수교육학원수             344872
직업기술학원수             344872
기타학원수               344872
번지_jm                    0
좌표X_jm                   0
좌표Y_jm                   0
도로명_nan               2339
도로명주소_nan             2339
번지주소_2                   0
도로명_fill               991
최단거리초등학교명                0
최단거리초등학교Km               0
좌표X_2                    0
좌표Y_2                    0
1번째_가까운_역_이름             0
1번째_가까운_역_호선             0
1번째_가까운_역_거리             0
1번째_가까운_역_도보시간           0
2번째_가까운_역_이름             0
2번째_가까운_역_호선             0
2번째_가까운_역_거리             0
2번째_가까운_역_도보시간           0
3번째_가까운_역_이름             0
3번째_가까운_역_호선             0
3

In [16]:
# submission data 분리 후 진행

# # 중복된 데이터 확인
# duplicates = df[df.duplicated()]

# # 결과 출력
# print("중복 데이터:")
# print(duplicates)

# # 중복된 데이터 삭제
# df.drop_duplicates(inplace=True)

중복 데이터:
             아파트명  전용면적(㎡)    계약년월  계약일   층  건축년도            도로명    target  \
201045   래미안블레스티지   59.967  202304   18  22  2019          선릉로 8  185000.0   
201051   래미안블레스티지   59.967  202305    8  16  2019          선릉로 8  190000.0   
201080       대치삼성   84.580  202306    2   8  2000      도곡로78길 22  220000.0   
201084       대치삼성   97.350  202306    3   4  2000      도곡로78길 22  240000.0   
201089       대치삼성   59.880  202306   29   2  2000      도곡로78길 22  162500.0   
...           ...      ...     ...  ...  ..   ...            ...       ...   
1127725      용마한신   59.220  202308   26   2  1990     사가정로71길 19       NaN   
1127869     삼성파크빌   84.920  202307   25  14  2006        을지로 221       NaN   
1127889      롯데캐슬   59.940  202308    9  23  2008       청계천로 400       NaN   
1127991      건영2차   84.960  202307   24  16  1996      신내로7나길 24       NaN   
1128030     라이프미성   59.600  202307   11   8  1992  용마산로129나길 101       NaN   

         is_test                       id       계약년월일  

In [ ]:
# 컬럼명에서 '(㎡)', 'k-', '(아파트,주상복합등등)', '(분양형태)' 삭제
def clean_column_name(name):
    name = re.sub(r'k-', '', name)
    name = re.sub(r'\(.*?\)', '', name)
    return name.strip()

# train과 test 데이터프레임의 컬럼명 정리
train.columns = [clean_column_name(col) for col in train.columns]
test.columns = [clean_column_name(col) for col in test.columns]

display(train.head(2))
display(test.head(2))

In [37]:
train.head(1)

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,등기신청일자,거래유형,중개사소재지,"k-단지분류(아파트,주상복합등등)",k-전화번호,k-팩스번호,단지소개기존clob,k-세대타입(분양형태),k-관리방식,k-복도유형,k-난방방식,k-전체동수,k-전체세대수,k-건설사(시공사),k-시행사,k-사용검사일-사용승인일,k-연면적,k-주거전용면적,k-관리비부과면적,k-전용면적별세대현황(60㎡이하),k-전용면적별세대현황(60㎡~85㎡이하),k-85㎡~135㎡이하,k-135㎡초과,k-홈페이지,k-등록일자,k-수정일자,고용보험관리번호,경비비관리형태,세대전기계약방법,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,8,3,1987,언주로 3,NaN,,-,-,아파트,025776611,025776673,NaN,분양,자치관리,계단식,개별난방,8.0,270.0,우성건설,모름,1987-11-21 00:00:00.0,22637.0,20204.0,22637.0,20.0,250.0,0.0,NaN,NaN,2022-11-09 20:10:43.0,2023-09-23 17:21:41.0,NaN,직영,단일계약,직영,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,124000


In [38]:
df.head(1)

,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,target,is_test,id,계약년월일,도로명주소,계약년,번호,k-아파트코드,k-아파트명,"k-단지분류(아파트,주상복합등등)",k-전체동수,k-전체세대수,k-건설사(시공사),준공일자,k-연면적,k-주거전용면적,k-관리비부과면적,k-전용면적별세대현황(60㎡이하),k-전용면적별세대현황(60㎡~85㎡이하),k-85㎡~135㎡이하,k-135㎡초과,주차대수,좌표X,좌표Y,출입구_x좌표_wgs84,출입구_y좌표_wgs84,지번주소_x,final_X,final_Y,Property_ID,X,Y,Nearest_Bus_Station_Distance,Nearest_Bus_Station_Name,Nearest_Subway_Station_Distance,Nearest_Subway_Station_Name,입주유형,단지명,동수_x,합계 세대수,분양 세대수,임대 세대수,토지 면적(㎡),합계 동수,5층 동수,5층 세대수,6~10층 동수,6~10층 세대수,11~15층 동수,11~15층 세대수,16~20층 동수,16~20층 세대수,21층이상 동수,21층이상 세대수,~40㎡ 세대수,40㎡~60㎡ 세대수,60㎡~85㎡ 세대수,85㎡~102㎡ 세대수,102㎡~135㎡ 세대수,135㎡~ 세대수,서울특별시_부동산소비심리지수,미분양_구,시점,기준금리,국고채,회사채,국고채-회사채,생산자물가지수,소비자물가지수,서울특별시_가계대출,서울특별시_주택담보대출,서울특별시_주택매매거래량,서울특별시_아파트매매거래량,전국매매가격지수,전국전세가격지수,서울특별시_주택인허가실적,서울특별시_주택착공실적,서울특별시_주택준공실적,서울특별시_주택미분양,미분양_서울특별시,공시가격기반단지명,동수_y,세대수,사용승인일자,소형학원수,중형학원수,대형학원수,종합학원수,입시학원수,외국어학원수,예체능학원수,독서실수,정보학원수,특수교육학원수,직업기술학원수,기타학원수,번지_jm,좌표X_jm,좌표Y_jm,도로명주소_nan,번지주소_2,최단거리초등학교명,최단거리초등학교Km
0,개포6차우성,79.97,201712.0,8.0,3.0,1987.0,언주로 3,124000.0,0.0,서울특별시 강남구 개포동 658-1 3층,2017-12-08,서울특별시 강남구 언주로 3,2017.0,40291.0,A10023565,개포6차우성아파트,아파트,8.0,270.0,우성건설,1987-11-21,22637.0,20204.0,22637.0,20.0,250.0,0.0,NaN,262.0,127.05721,37.476763,127.058091,37.476367,서울특별시 강남구 개포동 658-1,127.05721,37.476763,서울특별시 강남구 언주로 3,127.05721,37.476763,0.061744,개포우성아파트,1.129065,구룡,분양,6차우성,8.0,270.0,270.0,NaN,22539.0,8.0,8.0,270.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,250.0,NaN,NaN,NaN,94.2,0.0,201712.0,1.5,2.1,2.649,0.549,99.09,97.698,282438.7,176991.0,13740.0,8484.0,87.701,93.042,113131.0,13157.0,5968.0,45.0,45.0,개포6차우성아파트1동~8동,8.0,270.0,19871121.0,39.0,3.0,0.0,2.0,23.0,1.0,14.0,0.0,0.0,0.0,1.0,1.0,658-1,127.056859,37.476276,서울특별시 강남구 언주로 3,서울특별시 강남구 개포동 658-1,서울포이초등학교,0.385832


In [76]:
df.isna().sum()[125:]

좌표X_2               0
좌표Y_2               0
1번째_가까운_역_이름        0
1번째_가까운_역_호선        0
1번째_가까운_역_거리        0
1번째_가까운_역_도보시간      0
2번째_가까운_역_이름        0
2번째_가까운_역_호선        0
2번째_가까운_역_거리        0
2번째_가까운_역_도보시간      0
3번째_가까운_역_이름        0
3번째_가까운_역_호선        0
3번째_가까운_역_거리        0
3번째_가까운_역_도보시간      0
5분이하_역_개수           0
5분초과_10분이하_역_개수     0
10분초과_15분이하_역_개수    0
15분초과_20분이하_역_개수    0
dtype: int64

In [149]:
subway_train.columns

Index(['좌표X_2', '좌표Y_2', '1번째_가까운_역_이름', '1번째_가까운_역_호선', '1번째_가까운_역_거리',
       '1번째_가까운_역_도보시간', '2번째_가까운_역_이름', '2번째_가까운_역_호선', '2번째_가까운_역_거리',
       '2번째_가까운_역_도보시간', '3번째_가까운_역_이름', '3번째_가까운_역_호선', '3번째_가까운_역_거리',
       '3번째_가까운_역_도보시간', '5분이하_역_개수', '5분초과_10분이하_역_개수', '10분초과_15분이하_역_개수',
       '15분초과_20분이하_역_개수'],
      dtype='object')

In [584]:
data = df[['전용면적(㎡)', '층', '건축년도', 
         'target', 'is_test', '계약년월일',
         'k-전용면적별세대현황(60㎡이하)',
         'k-전용면적별세대현황(60㎡~85㎡이하)', 
         'k-85㎡~135㎡이하', 
         'k-135㎡초과', '주차대수', 'Nearest_Bus_Station_Name', 'Nearest_Subway_Station_Name',
 '5층 동수', '5층 세대수', '6~10층 동수', '6~10층 세대수', '11~15층 동수', '11~15층 세대수', '16~20층 동수', '16~20층 세대수',
 '21층이상 동수', '21층이상 세대수', '~40㎡ 세대수', '40㎡~60㎡ 세대수', '60㎡~85㎡ 세대수', '85㎡~102㎡ 세대수', '102㎡~135㎡ 세대수', 
 '135㎡~ 세대수', '시점','국고채', '회사채', '국고채-회사채', '생산자물가지수', '소비자물가지수', '서울특별시_가계대출',
 '서울특별시_주택담보대출', '서울특별시_주택매매거래량', '서울특별시_아파트매매거래량', '전국매매가격지수', '전국전세가격지수',
 '세대수', '좌표X_jm', '좌표Y_jm','번지주소_2', '도로명_fill', '최단거리초등학교명', '최단거리초등학교Km','좌표X_2', 
 '좌표Y_2', '1번째_가까운_역_이름', '1번째_가까운_역_호선', '1번째_가까운_역_거리', '1번째_가까운_역_도보시간', '2번째_가까운_역_이름', 
 '2번째_가까운_역_호선', '2번째_가까운_역_거리', '2번째_가까운_역_도보시간', '3번째_가까운_역_이름', '3번째_가까운_역_호선', 
 '3번째_가까운_역_거리', '3번째_가까운_역_도보시간', '5분이하_역_개수', '5분초과_10분이하_역_개수', '10분초과_15분이하_역_개수',
'15분초과_20분이하_역_개수']]

In [151]:
data.isna().sum()[:25]

아파트명                              2136
전용면적(㎡)                              0
층                                    0
건축년도                                 0
target                            9272
is_test                              0
계약년월일                                0
k-전용면적별세대현황(60㎡~85㎡이하)          263476
k-85㎡~135㎡이하                    263476
k-전용면적별세대현황(60㎡~85㎡이하)          263476
k-85㎡~135㎡이하                    263476
k-135㎡초과                       1127860
주차대수                            266282
Nearest_Bus_Station_Name             0
Nearest_Subway_Station_Name          0
5층 동수                          1086710
5층 세대수                         1082661
6~10층 동수                       1023845
6~10층 세대수                      1023845
11~15층 동수                       643203
11~15층 세대수                      643203
16~20층 동수                       803398
16~20층 세대수                      803398
21층이상 동수                        858287
21층이상 세대수                       858287
dtype: int64

In [152]:
df2 = data.copy()

In [153]:
df2 = df2[['번지주소_2', '아파트명']].copy()

In [154]:
import pandas as pd
import numpy as np


# 결측치를 같은 번지주소_2를 가진 행의 아파트명으로 채우기
df2['건물명'] = df2.groupby('번지주소_2')['아파트명'].transform(lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else np.nan))

# 결과 출력
df2


,번지주소_2,아파트명,건물명
0,서울특별시 강남구 개포동 658-1,개포6차우성,개포6차우성
1,서울특별시 강남구 개포동 658-1,개포6차우성,개포6차우성
2,서울특별시 강남구 개포동 658-1,개포6차우성,개포6차우성
3,서울특별시 강남구 개포동 658-1,개포6차우성,개포6차우성
4,서울특별시 강남구 개포동 658-1,개포6차우성,개포6차우성
...,...,...,...
1128089,서울특별시 중랑구 신내동 816,신내우디안1단지,신내우디안1단지
1128090,서울특별시 중랑구 신내동 816,신내우디안1단지,신내우디안1단지
1128091,서울특별시 중랑구 신내동 816,신내우디안1단지,신내우디안1단지
1128092,서울특별시 중랑구 신내동 816,신내우디안1단지,신내우디안1단지


In [168]:
df2[df2['아파트명'].isna()][:50]

,번지주소_2,아파트명,건물명
3839,서울특별시 강남구 역삼동 828-21,NaN,NaN
3840,서울특별시 강남구 역삼동 828-21,NaN,NaN
3841,서울특별시 강남구 역삼동 828-21,NaN,NaN
3842,서울특별시 강남구 역삼동 828-76,NaN,NaN
18113,서울특별시 관악구 봉천동 685-103,NaN,NaN
18114,서울특별시 관악구 봉천동 685-103,NaN,NaN
18115,서울특별시 관악구 봉천동 685-103,NaN,NaN
25390,서울특별시 구로구 구로동 739-29,NaN,NaN
25391,서울특별시 구로구 구로동 739-29,NaN,NaN
25392,서울특별시 구로구 구로동 740-33,NaN,NaN


In [171]:
# 특정 조건에 맞는 행에서 결측치를 '뜨라네'로 채우기
condition = df2['번지주소_2'].str.contains('서울특별시 강남구 역삼동 828-21')
df2.loc[condition, '건물명'] = df2.loc[condition, '건물명'].fillna('뜨라네')

In [522]:
df2[df2['건물명'].isna()]['번지주소_2'].value_counts()

서울특별시 구로구 구로동 770-8       6
서울특별시 구로구 구로동 794-32      5
서울특별시 구로구 구로동 742-68      5
서울특별시 구로구 구로동 739-19      5
서울특별시 구로구 구로동 748-32      5
                         ..
서울특별시 구로구 구로동 743-14      1
서울특별시 구로구 구로동 794-24      1
서울특별시 구로구 구로동 743-52      1
서울특별시 구로구 구로동 806-40      1
서울특별시 동대문구 이문동 257-516    1
Name: 번지주소_2, Length: 116, dtype: int64

In [524]:
# 특정 조건에 맞는 행에서 결측치를 '뜨라네'로 채우기
condition = df2['번지주소_2'].str.contains('서울특별시 구로구 구로동 744-16')
df2.loc[condition, '건물명'] = df2.loc[condition, '건물명'].fillna('1')

In [523]:
df2['건물명'].isna().sum()

350

In [541]:
df2.loc[df2['건물명'].isna(), '건물명'] = '1'

In [543]:
df2[df2['아파트명'].isna()]

,번지주소_2,아파트명,건물명
3839,서울특별시 강남구 역삼동 828-21,NaN,뜨라네
3840,서울특별시 강남구 역삼동 828-21,NaN,뜨라네
3841,서울특별시 강남구 역삼동 828-21,NaN,뜨라네
3842,서울특별시 강남구 역삼동 828-76,NaN,뜨라네
18113,서울특별시 관악구 봉천동 685-103,NaN,보람아파트
...,...,...,...
1122857,서울특별시 동대문구 장안동 404-13,NaN,아이엔에스새터아파트
1123271,서울특별시 동작구 상도동 323-4,NaN,상진빌딩
1126863,서울특별시 영등포구 대림동 1101-1,NaN,유탑유블레스
1126864,서울특별시 영등포구 대림동 1101-1,NaN,유탑유블레스


In [548]:
df2[['건물명']].to_csv(f'{data_path}b_name.csv', index=False)

In [585]:
biul = pd.read_csv(f'{data_path}b_name.csv')

In [588]:
data = pd.concat([data, biul], axis=1)

In [590]:
data[data['건물명'] == '1'] = '0'

In [554]:
# data['건물명'] = df2['건물명']

In [591]:
data.drop(columns=['아파트명'], inplace=True)

In [592]:
data.isna().sum()[:25]

전용면적(㎡)                              0
층                                    0
건축년도                                 0
target                            9272
is_test                              0
계약년월일                                0
k-전용면적별세대현황(60㎡~85㎡이하)          263128
k-85㎡~135㎡이하                    263128
k-85㎡~135㎡이하                    263128
k-135㎡초과                       1127510
주차대수                            265934
Nearest_Bus_Station_Name             0
Nearest_Subway_Station_Name          0
5층 동수                          1086365
5층 세대수                         1082316
6~10층 동수                       1023497
6~10층 세대수                      1023497
11~15층 동수                       642855
11~15층 세대수                      642855
16~20층 동수                       803048
16~20층 세대수                      803048
21층이상 동수                        857937
21층이상 세대수                       857937
~40㎡ 세대수                       1016528
40㎡~60㎡ 세대수                     616993
dtype: int64

In [593]:
data.columns

Index(['전용면적(㎡)', '층', '건축년도', 'target', 'is_test', '계약년월일',
       'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-85㎡~135㎡이하', 'k-135㎡초과',
       '주차대수', 'Nearest_Bus_Station_Name', 'Nearest_Subway_Station_Name',
       '5층 동수', '5층 세대수', '6~10층 동수', '6~10층 세대수', '11~15층 동수', '11~15층 세대수',
       '16~20층 동수', '16~20층 세대수', '21층이상 동수', '21층이상 세대수', '~40㎡ 세대수',
       '40㎡~60㎡ 세대수', '60㎡~85㎡ 세대수', '85㎡~102㎡ 세대수', '102㎡~135㎡ 세대수',
       '135㎡~ 세대수', '시점', '국고채', '회사채', '국고채-회사채', '생산자물가지수', '소비자물가지수',
       '서울특별시_가계대출', '서울특별시_주택담보대출', '서울특별시_주택매매거래량', '서울특별시_아파트매매거래량',
       '전국매매가격지수', '전국전세가격지수', '세대수', '좌표X_jm', '좌표Y_jm', '번지주소_2', '도로명_fill',
       '최단거리초등학교명', '최단거리초등학교Km', '좌표X_2', '좌표Y_2', '1번째_가까운_역_이름',
       '1번째_가까운_역_호선', '1번째_가까운_역_거리', '1번째_가까운_역_도보시간', '2번째_가까운_역_이름',
       '2번째_가까운_역_호선', '2번째_가까운_역_거리', '2번째_가까운_역_도보시간', '3번째_가까운_역_이름',
       '3번째_가까운_역_호선', '3번째_가까운_역_거리', '3번째_가까운_역_도보시간', '5분이하_역_개수',
       '5분초과_10분이하_역_개수', '10분초과_15분이하_역_개수', '15분초과_20분이하_역

In [598]:
data[data['k-전용면적별세대현황(60㎡~85㎡이하)'].isna()]['k-전용면적별세대현황(60㎡~85㎡이하)'] = 0

In [601]:
data[data['k-전용면적별세대현황(60㎡~85㎡이하)']==0]['k-전용면적별세대현황(60㎡~85㎡이하)'].count()

126356